In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *

VISUALIZE = False
graph = None
SAMPLED_DATA = defaultdict(dict)
UPDATE_DAT = True

GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
DATA_PATH = "./data"

gtimer = GlobalTimer.instance()
elog = Logger()
crob = CombinedRobot(connection_list=(False, False))

CHECK_DICT = {}

# ## Load Global params
DATASET_LIST = sorted(filter(lambda x: x!="backup", os.listdir(DATA_PATH)))

connection_list
(False, False)


## dataset

In [2]:
DATASET = DATASET_LIST[0]
CHECK_DICT[DATASET] = {}
CURRENT_PATH = os.path.join(DATA_PATH, DATASET)

## Load global params
GLOBAL_PARAMS = load_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME))
WDH = GLOBAL_PARAMS["WDH"]
L_CELL = GLOBAL_PARAMS["L_CELL"]
RATIO_DIMS = GLOBAL_PARAMS["RATIO_DIMS"]
REACH_OFFSET_DICT = GLOBAL_PARAMS["REACH_OFFSET_DICT"]
GRIPPER_REFS = GLOBAL_PARAMS["GRIPPER_REFS"]
BASE_LINK = GLOBAL_PARAMS["BASE_LINK"]
S_F_RATIO = GLOBAL_PARAMS["S_F_RATIO"]
TIMEOUT = GLOBAL_PARAMS["TIMEOUT"]

CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
Ws, Ds, Hs = WDH
Nw, Nd, Nh = Nwdh = int(Ws / L_CELL), int(Ds / L_CELL), int(Hs / L_CELL)
L_MAX = L_CELL * RATIO_DIMS
print("scene size: {} ({},{},{})".format(Nw * Nd * Nh, Nw, Nd, Nh))

# ## Load world
WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))

scene size: 3375 (15,15,15)


## world

In [3]:
WORLD = WORLD_LIST[0]
CHECK_DICT[DATASET][WORLD] = {}
WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
SAMPLED_DATA["WORLD"] = load_json(os.path.join(WORLD_PATH, WORLD_FILENAME))
Trbt_dict = SAMPLED_DATA["WORLD"]["Trbt_dict"]
reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

cam = None
# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict,
                                                                   crob.custom_limits, start_rviz=VISUALIZE)
ghnd = GeometryHandle(urdf_content)
time.sleep(2)

# set graph
graph = ConstraintGraph(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                        urdf_content=urdf_content, combined_robot=crob)
graph.set_camera(cam)
graph.set_cam_robot_collision(_add_cam_poles=False, color=(1, 1, 0, 0.3))
if VISUALIZE: graph.set_rviz()

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(graph)
ui_broker.start_server()

# set rviz
if VISUALIZE: graph.set_rviz(crob.home_pose)
ui_broker.set_tables()

for gripper in GRIPPER_REFS.values():
    graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0, 0, 0],
                          link_name=gripper['link_name'])
graph.register_binder(name='base', _type=PlaceFrame, point=[0, 0, 0], rpy=[0, 0, 0], link_name=BASE_LINK)
vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                              dims=(0, 0, 0), center=(0, 0, 0), rpy=(0, 0, 0), collision=False, display=False
                              )
graph.add_object("virtual",
                 SingleHandleObject(_object=vtem,
                                    action_point=FramePoint(name="point", _object=vtem, point=(0, 0, 0),
                                                            rpy=(0, 0, 0), name_full=None)),
                 binding=("point", "base"))

obj_list = []
col_obj_list = []

# ## Load scene
SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## ScENE

In [4]:
SCENE = SCENE_LIST[0]
CHECK_DICT[DATASET][WORLD][SCENE] = {}
SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
SAMPLED_DATA["SCENE"] = load_json(os.path.join(SCENE_PATH, SCENE_FILENAME))
Q_s = np.array(SAMPLED_DATA["SCENE"]["Q_s"])
Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph, Q_s_loaded=Q_s)
Q_s_dict = SAMPLED_DATA["SCENE"]["Q_s_dict"]
obj_dat = SAMPLED_DATA["SCENE"]["obj_dat"]

if VISUALIZE:
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
for obj in obj_list: graph.remove_geometry(obj)
for odat in obj_dat:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], \
                                            odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="{}_{}_{}_{}".format(gtype.name, *nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
    obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

for obj in col_obj_list: graph.remove_geometry(obj)

if VISUALIZE: graph.set_rviz()
dcol = DataCollector(graph, GRIPPER_REFS, S_F_RATIO=S_F_RATIO)
if VISUALIZE: graph.set_rviz()

# planners
mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path,
                      urdf_content=graph.urdf_content,
                      robot_names=graph.combined_robot.robot_names,
                      binder_links=[v.object.link_name for v in graph.binder_dict.values()],
                      ghnd=graph.ghnd)
dual_mplan_dict = get_dual_planner_dict(GRIPPER_REFS, graph.ghnd, graph.urdf_content, graph.urdf_path,
                                        graph.link_names, graph.combined_robot.robot_names)

# ## Load action
ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/tamp/.local/lib/python2.7/site-packages/dash/dash.py", line 1716, in run_server
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/home/tamp/.local/lib/python2.7/site-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/home/tamp/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1052, in run_simple
    inner()
  File "/home/tamp/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1005, in inner
    fd=fd,
  File "/home/tamp/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 848, in make_server
    host, port, app, request_handler, passthrough_errors, ssl_context, fd=fd
  File "/home/tamp/.local/lib/python2.7/s

## initialize

In [5]:
link_names = graph.link_names
joint_names = crob.joint_names
joint_num = crob.joint_num
IGNORE_CTEMS = ["panda1_hand_Cylinder_1", 'panda1_hand_Cylinder_2']
joint_index_dict = {joint.name:None for joint in urdf_content.joints}
joint_index_dict.update({jname:idx for idx, jname in zip(range(joint_num), joint_names)})
centers = get_centers(Nwdh, L_CELL)
merge_pairs = get_merge_pairs(ghnd)
merge_paired_ctems(graph=graph, merge_pairs=merge_pairs, VISUALIZE=VISUALIZE)
for cname in IGNORE_CTEMS:
    graph.remove_geometry(graph.ghnd.NAME_DICT[cname])

## calc joint kinematics

In [6]:
gtimer.reset()
gtimer.tic("test_links")
Tlink_dict = {}
chain_dict = {}
Tj_arr = np.zeros((joint_num, 4, 4))
Tj_inv_arr = np.zeros((joint_num, 4, 4))
joint_axis_arr = np.zeros((joint_num,3))

gtimer.tic("T_chain")
Tlink_dict = build_T_chain(link_names, Q_s_dict, urdf_content)
gtimer.toc("T_chain")

for lname in link_names:
    gtimer.tic("get_chain")
    jnames = filter(lambda x: x in joint_names, 
                    urdf_content.get_chain(root=BASE_LINK, tip=lname, joints=True, links=False))
    gtimer.toc("get_chain")
    chain_dict[lname] = [1 if pj in jnames else 0 for pj in joint_names]
gtimer.toc("test_links")

gtimer.tic("test_joints")
for i_j, jname in zip(range(joint_num), joint_names):
    joint = urdf_content.joint_map[jname]
    lname = joint.child
    Tj_arr[i_j,:,:] = Tlink_dict[lname]
    Tj_inv_arr[i_j,:,:] = SE3_inv(Tlink_dict[lname])
    joint_axis_arr[i_j] = joint.axis
gtimer.toc("test_joints")
print(gtimer)

test_links: 	3.0 ms/1 = 3.179 ms (3.179/3.179)
T_chain: 	2.0 ms/1 = 2.285 ms (2.285/2.285)
get_chain: 	0.0 ms/21 = 0.009 ms (0.004/0.016)
test_joints: 	1.0 ms/1 = 0.716 ms (0.716/0.716)



## calc cell

In [7]:
gtimer.reset()
gtimer.tic("calc_cell")
__p = centers.reshape(Nwdh+(1,3)) - Tj_arr[:,:3,3].reshape((1,1,1,joint_num,3))
__w = np.sum(Tj_arr[:,:3,:3] * joint_axis_arr.reshape(joint_num, 1, 3), axis=-1).reshape((1,1,1,joint_num,3))
__w = np.tile(__w, Nwdh+(1,1))
__v = np.cross(__w, __p)
xi = np.concatenate([__w, __v],axis=-1)
gtimer.toc("calc_cell")

4.6539306640625

## calc ctems

In [57]:
gtimer.tic("calc_ctems")
ctem_names, ctem_TFs, ctem_dims, ctem_cells, ctem_links, ctem_joints, ctem_types = \
    defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list), dict()
for ctem in ghnd:
    if not ctem.collision:
        continue 
    key = gtype_to_otype(ctem.gtype).name
    ctem_names[key].append(ctem.name)
    Ttem = np.matmul(Tlink_dict[ctem.link_name], ctem.Toff)
    ctem_TFs[key].append(Ttem)
    ctem_dims[key].append(ctem.dims)
    ctem_cells[key].append(get_cell(Ttem[:3,3], L_CELL, Nwdh))
    ctem_links[key].append(ctem.link_name)
    ctem_joints[key].append(chain_dict[ctem.link_name])
    ctem_types[key] = ctem.gtype

    
verts_dict = {}
for key_cur in ctem_cells:
    ctem_TFs[key_cur] = np.array(ctem_TFs[key_cur])
    cell_array = np.array(ctem_cells[key_cur])
    all_rearranged = False
    while not all_rearranged:
        all_rearranged = True
        for cell in cell_array:
            idxset = np.where(np.all(cell_array==cell, axis=-1))[0]
            if len(idxset)>1:
                all_rearranged = False
                cell_array = rearrange_cell_array(cell_array, idxset, L_CELL, ctem_TFs[key_cur], centers)
                break
    ctem_cells[key_cur] = cell_array
    gtype = ctem_types[key_cur]
    TFs = ctem_TFs[key_cur]
    dims = np.array(ctem_dims[key_cur])
    default_vert = DEFAULT_VERT_DICT[gtype]
    verts_dim = default_vert.reshape((1,-1,3))*dims.reshape((-1,1,3))
    cell_array = ctem_cells[key_cur]
    verts = np.zeros_like(verts_dim)
    for iv in range(verts.shape[1]):
        verts[:,iv,:] = matmul_md(TFs[:,:3,:3], verts_dim[:,iv,:,np.newaxis])[:,:,0]+TFs[:,:3,3]
    verts_loc = (verts - (cell_array[:,np.newaxis,:]*L_CELL+L_CELL/2))
    verts_dict[key_cur] = verts_loc
gtimer.toc("calc_ctems")

224.67708587646484

## Action

In [ ]:
ACTION = ACTION_LIST[0]
print("[BEGIN] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
CHECK_DICT[DATASET][WORLD][SCENE][ACTION] = {}
snode_dict_bak = {int(k): v for k, v in load_json(os.path.join(SCENE_PATH, ACTION)).items()}
dcol.snode_dict = dcol.manager.dict()
for k, v in snode_dict_bak.items():
    dcol.snode_dict[k] = deepcopy(v)
snode_keys = sorted(snode_dict_bak.keys())

if snode_keys:
    sk0 = snode_keys[0]
    rname, tar, inhand_coll, obj_coll = \
        snode_dict_bak[sk0]["rname1"], snode_dict_bak[sk0]["rname2"], \
        snode_dict_bak[sk0]["obj1"]["collision"], snode_dict_bak[sk0]["obj2"]["collision"]
else:
    print("[END] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
    print("No Item")
    continue

dcol.check_loop_mp(test_full_mp, GRIPPER_REFS=GRIPPER_REFS,  Q_s=Q_s, dual_mplan_dict=dual_mplan_dict, mplan=mplan, N_retry=N_retry_test, timeout=TIMEOUT)
succ_old_list = np.array([snode_dict_bak[skey]["success"] for skey in snode_keys])
succ_now_list = np.array([dcol.snode_dict[skey]["succ_count"]>0 for skey in snode_keys])

if rname and tar:  # handover case
    action_type = "HANDOVER"
elif inhand_coll:  # place case
    action_type = "PLACE"
elif obj_coll:  # pick case
    action_type = "PICK"
else:
    raise (RuntimeError("non-implemented case"))

check_results = succ_old_list==succ_now_list
print(succ_old_list)
print(succ_now_list)
CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["action_type"] = action_type
CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["check_ratio"] = np.mean(check_results)
CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["succ_corrects"] = np.mean(
    check_results[np.where(succ_old_list)])
CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["fail_corrects"] = np.mean(
    check_results[np.where(np.logical_not(succ_old_list))])
CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["num"] = len(check_results)

if UPDATE_DAT:
    for skey in snode_keys:
        if not snode_dict_bak[skey]["success"] and dcol.snode_dict[skey]["success"]:
            snode_dict_bak[skey]["success"] = dcol.snode_dict[skey]["success"]
    save_json(os.path.join(SCENE_PATH, ACTION), snode_dict_bak)

if VISUALIZE: graph.set_rviz()

print("[END] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
elog.log(CHECK_DICT[DATASET][WORLD][SCENE][ACTION], "{}-{}".format(SCENE, ACTION), print_now=False)
for k, v in CHECK_DICT[DATASET][WORLD][SCENE][ACTION].items():
    print("{}: {}".format(k, str(v)))

In [2]:
import os
from pkg.utils.utils import *

In [4]:
GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
DATA_PATH = "./data"

In [14]:
count_all = 0
DATASET_LIST = sorted(filter(lambda x: x!="backup", os.listdir(DATA_PATH)))
for DATASET in DATASET_LIST:
    CURRENT_PATH = os.path.join(DATA_PATH, DATASET)
    WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))
    for WORLD in WORLD_LIST:
        WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
        SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
        for SCENE in SCENE_LIST:
            SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
            ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
            for ACTION in ACTION_LIST:
                snode_dict_bak = {int(k): v for k, v in load_json(os.path.join(SCENE_PATH, ACTION)).items()}
                print(len(snode_dict_bak))
                count_all += len(snode_dict_bak)

4
4
7
7
12
11
58
28
41
67
19
15
8
2
6
97
33
4
1
15
17
14
11
22
0
9
0
49
5
26
2
18
29
0
8
4
0
20
6
20
18
28
31
22
21
75
31
19
9
25
14
17
6
8
69
14
20
102
32
19
21
21
22
7
13
16
8
30
17
46
42
23
40
8
11
34
12
14
12
17
7
47
26
17
3
11
7
30
29
4
35
17
25
7
15
2
0
21
16
39
31
19
43
16
39
52
30
12
4
16
24
19
21
22
46
7
16
52
13
27
57
33
18
41
32
6
16
14
15
0
27
4
3
4
11
46
20
17
43
23
25
43
19
17
44
19
16
50
22
24
42
32
16
60
35
11
38
28
8
17
9
15
0
11
10
67
26
22
34
18
10
49
3
33
48
43
20
0
8
9
54
33
24
77
26
8
26
17
20
17
22
10
8
17
10
34
14
15
27
8
6
16
20
14
13
22
20
37
34
44
77
10
7
28
21
21
27
3
5
18
8
10
20
9
4
70
28
16
16
20
21
9
1
30
80
24
19
68
22
26
4
22
16
0
23
18
30
29
20
26
20
12
55
5
28
22
35
14
42
19
19
8
13
17
12
44
24
10
9
15
7
15
12
14
13
7
6
32
13
38
14
27
62
22
13
13
16
29
62
16
18
54
13
15
85
18
9
0
13
6


In [15]:
count_all

6675